In [1]:
%matplotlib inline

Formalizing Data Processing
=======================================

**Suhas Somnath**

9/8/2017

**In this example, we will learn how to write a simple yet formal pycroscopy class for processing data.**

Introduction
------------
Most of code written for scientific research is in the form of single-use / one-off scripts due to a few common reasons:
* the author feels that it is the fastest mode to accomplishing a research task 
* the author feels that they are unlikely to perform the same operation again 
* the author does not anticipate the possibility that others may need to run their code

However, more often than not, nearly all researchers have found that one or more of these assumptions fail and a lot of time is spent on fixing bugs and generalizing / formalizing code such that it can be shared or reused. Moreover, we live in an era of open science where the scientific community and an ever-increasing number of scientific journals are moving towards a paradigm where the data and code need to be made available with journal papers. Therefore, in the interest of saving time, energy, and reputation (you do not want to show ugly code / data. Instead you want to be the paragon of clean intellgible data and code), it makes a lot more sense to formalize (parts of) one's data analysis code.  

For many researchers, formalizing data processing or analysis may seem like a daunting task due to the complexity of and the number of sub-operations that need to performed. **pycroscopy.Process** greatly simplifies the process of formalizing code by lifting the burden of memory management, CPU management, parallel computing, interruption handling, etc. off the user. The user only needs to address the operation that needs to be performed on each chunk of data along with the data reading and writing.

Data processing / analysis using **pycroscopy.Process** typically only involves a few basic tasks:
1. Reading data from file
2. Computation
3. Writing results to disk

This example is based on the parallel computing primer where we fit a dataset containing spectra at each location to a
function. While that example focused on comparing serial and parallel computing, we will focus on demonstrating the simplicity with which such a data analysis algorithm can be formalized. 

This example is a simplification of the pycroscopy.analysis.BESHOFitter class. 

Import necessary packages
-------------------------

In [1]:
from __future__ import division, print_function, absolute_import, unicode_literals

# The package for accessing files in directories, etc.:
import os

# Warning package in case something goes wrong
from warnings import warn

# Package for downloading online files:
try:
    # This package is not part of anaconda and may need to be installed.
    import wget
except ImportError:
    warn('wget not found.  Will install with pip.')
    import pip
    pip.main(['install', 'wget'])
    import wget

# The mathematical computation package:
import numpy as np
from numpy import exp, abs, sqrt, sum, real, imag, arctan2, append

# The package used for creating and manipulating HDF5 files:
import h5py

# Packages for plotting:
import matplotlib.pyplot as plt

# Finally import pycroscopy for certain scientific analysis:
if True:
    import sys
    sys.path.append(os.path.split(os.path.abspath('.'))[0])
    import pycroscopy as px
else:
    try:
        import pycroscopy as px
    except ImportError:
        warn('pycroscopy not found.  Will install with pip.')
        import pip
        pip.main(['install', 'pycroscopy'])
        import pycroscopy as px

//anaconda/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/syz/PycharmProjects/pycroscopy/pycroscopy/__init__.py:25: UserWarning: You are using the unity_dev branch, which is aimed at a 1.0 release for pycroscopy. Be advised - this branch changes very significantly and frequently. Use the master or dev branches for regular purposes.
  warn('You are using the unity_dev branch, which is aimed at a 1.0 release for pycroscopy. '


pycroscopy.Process
============

Every process class consists of the same basic functions:
* **__init__** - instantiates a 'Process' object of this class after validating the inputs.
    * In this case pycroscopy.Process takes are of most of the checking - whether h5_main is indeed a Main dataset, etc.
* **_create_results_datasets** - creates the HDF5 datasets and Group(s) to store the results.
* **_map_function** - the operation that will per be performed on each element in the dataset.
    * In this case, we will use the wavelet_peaks() function and then add some post-processing of the results
* **test** - This function lets the user test the **map_function** on a unit of data (a single pixel / spectra in this case) to see if it returns the desired results. 
    * This function is generally very simple to implement
    * it saves a lot of computational time by allowing the user to spot-check results before computing on the entire dataset
* **_write_results_chunk** - writes the computed results back to the file

Note that:

* Only the code specific to this process needs to be implemented. The generic portions common to most
  Processes will be handled by the Process class.
* The other functions such as the sho_function, sho_fast_guess function are all specific to this process. These have
  been inherited directly from the BE SHO model.
* While the class appears to be large, remember that the majority of it deals with the creation of the datasets to store
  the results and the actual function that one would have anyway regardless of serial / parallel computation of the
  function. The additional code to turn this operation into a Pycroscopy Process is actually rather minimal. As
  described earlier, the goal of the Process class is to modularize and compartmentalize the main sections of the code
  in order to facilitate faster and more robust implementation of data processing algorithms.

In [2]:
class PeakFinder(px.Process):

    def __init__(self, h5_main, cores=None):
        """
        Validate the inputs and set some parameters

        Parameters
        ----------
        h5_main - dataset to compute on
        cores - Number of CPU cores to use for computation - Optional
        """
        super(PeakFinder, self).__init__(h5_main, cores=cores)
        self.process_name = 'Peak_Finding'
        
    def test(self, pixel_ind):
        """
        
        Parameters
        ----------
        """
        spectra = self.h5_main[pixel_ind]
        func = self._map_function
        return func(spectra)

    def _create_results_datasets(self):
        """
        Creates the datasets an Groups necessary to store the results.
        There are only TWO operations happening in this function:
        1. Creation of HDF5 group to hold results
        2. Creation of a HDF5 dataset to hold results
        
        Please see examples on utilities for writing pycroscopy HDF5 files for more information
        """
        # create a HDF5 group to hold the results
        h5_results_group = px.hdf_utils.create_results_group(self.h5_main, self.process_name)
        
        # Explicitely stating all the inputs to write_main_dataset
        # The process will reduce the spectra at each position to a single value
        # Therefore, the result is a 2D dataset with the same number of positions as self.h5_main
        results_shape = (self.h5_main.shape[0], 1)
        results_dset_name = 'Peak_Response'
        results_quantity = 'Amplitude'
        results_units = 'V'
        pos_dims = None # Reusing those linked to self.h5_main
        spec_dims = px.write_utils.Dimension('Empty', 'a. u.', 1)
        
        
        # Create an empty results dataset that will hold all the results
        self.h5_results = px.hdf_utils.write_main_dataset(h5_results_group, results_shape, results_dset_name, 
                                                          results_quantity, results_units, pos_dims, spec_dims, 
                                                          dtype=np.float32,
                                                          h5_pos_inds=self.h5_main.h5_pos_inds,
                                                          h5_pos_vals=self.h5_main.h5_pos_vals)
        # Note that this function automatically creates the ancillary datasets and links them.
        
        print('Finshed creating datasets')
    
    def _write_results_chunk(self):
        """
        Write the computed results back to the H5
        In this case, there isn't any more additional post-processing required
        """            
        # write the results to the file 
        self.h5_results[:, 0] = np.array(self._results)
        
        # Flush the results to ensure that they have indeed been written to the file
        self.h5_main.file.flush()

        # Now update the start position
        self._start_pos = self._end_pos
        # this should stop the computation.        

    @staticmethod
    def _map_function(spectra, *args, **kwargs):
        """
        As in typical scientific problems, the results from wavelet_peaks() need to be
        post-processed
        
        In this case, the wavelet_peaks() function can sometimes return 0 or more than one peak
        for spectra that are very noisy
        
        Knowing that the peak is typically at the center of the spectra,
        we return the central index when no peaks were found
        Or the index closest to the center when multiple peaks are found
        
        Finally once we have a single index, we need to index the spectra by that index
        in order to get the amplitude at that frequency.
        """
        
        peak_inds = px.analysis.guess_methods.GuessMethods.wavelet_peaks(spectra, peak_widths=[20, 60], peak_step=30)
        
        central_ind = len(spectra) // 2
        if len(peak_inds) == 0:
            # too few peaks
            # set peak to center of spectra
            val = central_ind
        elif len(peak_inds) > 1:
            # too many peaks
            # set to peak closest to center of spectra
            dist = np.abs(peak_inds - central_ind)
            val = peak_inds[np.argmin(dist)]
        else:
            # normal situation
            val = peak_inds[0]
        return np.abs(spectra[val])

Load the dataset
================

For this example, we will be working with a Band Excitation Piezoresponse Force Microscopy (BE-PFM) imaging dataset
acquired from advanced atomic force microscopes. In this dataset, a spectra was collected for each position in a two
dimensional grid of spatial locations. Thus, this is a three dimensional dataset that has been flattened to a two
dimensional matrix in accordance with the pycroscopy data format.



In [3]:
# download the raw data file from Github:
h5_path = 'temp.h5'
url = 'https://raw.githubusercontent.com/pycroscopy/pycroscopy/master/data/BELine_0004.h5'
if os.path.exists(h5_path):
    os.remove(h5_path)
_ = wget.download(url, h5_path, bar=None)

In [4]:
# Open the file in read-only mode
h5_file = h5py.File(h5_path, mode='r+')

In [5]:
h5_meas_grp = h5_file['Measurement_000']

# Accessing the dataset of interest:
h5_main = px.PycroDataset(h5_meas_grp['Channel_000/Raw_Data'])
print('\nThe main dataset:\n------------------------------------')
print(h5_main)

num_rows, num_cols = h5_main.pos_dim_sizes


The main dataset:
------------------------------------
<HDF5 dataset "Raw_Data": shape (16384, 119), type "<c8">
located at: 
/Measurement_000/Channel_000/Raw_Data 
Data contains: 
[''] (['a']) 
Data dimensions and original shape: 
Position Dimensions: 
X - size: 128 
Y - size: 128 
Spectroscopic Dimensions: 
Frequency - size: 119


In [6]:
fitter = PeakFinder(h5_main)

Consider calling test() to check results before calling compute() which computes on the entire dataset and writes back to the HDF5 file


In [7]:
fitter.test(15)

0.0048981714

In [8]:
h5_results = fitter.compute(override=True)

Finshed creating datasets
You can abort this computation at any time and resume at a later time!
	If you are operating in a python console, press Ctrl+C or Cmd+C to abort
	If you are in a Jupyter notebook, click on "Kernel">>"Interrupt"
Finished parallel computation
[0.00200135 0.00322685 0.00272113 0.00312234 0.00257732 0.00305148
 0.00268378 0.00300558 0.00290046 0.00285854 0.00274262 0.00277178
 0.00440468 0.00514442 0.00387373 0.00489817 0.00516523 0.00460631
 0.00151064 0.00235155 0.00232911 0.00010282 0.00141946 0.00056969
 0.00288982 0.00265272 0.00141847 0.00286329 0.00421067 0.00333027
 0.00097382 0.00137351 0.00066657 0.00028706 0.00070315 0.0007369
 0.00050666 0.00169736 0.00175795 0.00014987 0.00144386 0.00425164
 0.00322629 0.0014094  0.00093455 0.00073421 0.00121491 0.00117978
 0.00135916 0.00373153 0.00655445 0.00656324 0.00810891 0.00848985
 0.0075443  0.00756923 0.00507225 0.00301273 0.00027978 0.0012216
 0.00132498 0.00134131 0.0017612  0.00189815 0.00168737 0.0011911

In [ ]:
row_ind, col_ind = 103, 19
pixel_ind = col_ind + row_ind * num_cols
spectra = h5_main[pixel_ind]

peak_inds = wavelet_peaks(spectra, peak_widths=[20, 60], peak_step=30)

fig, axis = plt.subplots()
axis.scatter(np.arange(len(spectra)), np.abs(spectra), c='black')
axis.axvline(peak_inds[0], color='r', linewidth=2)
axis.set_ylim([0, 1.1 * np.max(np.abs(spectra))]);
axis.set_title('wavelet_peaks found peaks at index: {}'.format(peak_inds), fontsize=16)

In [ ]:
# Open the file in read-only mode
h5_file = h5py.File(h5_path, mode='r+')

# Get handles to the the raw data along with other datasets and datagroups that contain necessary parameters
h5_meas_grp = h5_file['Measurement_000']
num_rows = px.hdf_utils.get_attr(h5_meas_grp, 'grid_num_rows')
num_cols = px.hdf_utils.get_attr(h5_meas_grp, 'grid_num_cols')

# Getting a reference to the main dataset:
h5_main = h5_meas_grp['Channel_000/Raw_Data']

# Extracting the X axis - vector of frequencies
h5_spec_vals = px.hdf_utils.getAuxData(h5_main, 'Spectroscopic_Values')[-1]
freq_vec = np.squeeze(h5_spec_vals.value) * 1E-3

Use the ShoGuess class, defined earlier, to calculate the four
parameters of the complex gaussian.



In [ ]:
fitter = ShoGuess(h5_main, cores=1)
h5_results_grp = fitter.compute()
h5_guess = h5_results_grp['Guess']

row_ind, col_ind = 103, 19
pix_ind = col_ind + row_ind * num_cols
resp_vec = h5_main[pix_ind]
norm_guess_parms = h5_guess[pix_ind]

# Converting from compound to real:
norm_guess_parms = px.io_utils.compound_to_scalar(norm_guess_parms)
print('Functional fit returned:', norm_guess_parms)
norm_resp = px.be_sho.SHOfunc(norm_guess_parms, freq_vec)

Plot the Amplitude and Phase of the gaussian versus the raw data.



In [ ]:
fig, axes = plt.subplots(nrows=2, sharex=True, figsize=(5, 10))
for axis, func, title in zip(axes.flat, [np.abs, np.angle], ['Amplitude (a.u.)', 'Phase (rad)']):
    axis.scatter(freq_vec, func(resp_vec), c='red', label='Measured')
    axis.plot(freq_vec, func(norm_resp), 'black', lw=3, label='Guess')
    axis.set_title(title, fontsize=16)
    axis.legend(fontsize=14)

axes[1].set_xlabel('Frequency (kHz)', fontsize=14)
axes[0].set_ylim([0, np.max(np.abs(resp_vec)) * 1.1])
axes[1].set_ylim([-np.pi, np.pi])

**Delete the temporarily downloaded file**



In [ ]:
h5_file.close()
os.remove(h5_path)